In [0]:
from pyspark.sql.functions import *

In [0]:
# Define variables
scope = "izanabees-kv"
key = "KEY-ADLS"
storage_account_name = "izanabees"
container_name = "bees"
mount_point = '/mnt/bees'

# Check if mount point is already mounted
if not any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
  
  # Get the connection string from secrets
  connect_str = dbutils.secrets.get(scope=scope, key=key)

  # Mount the container
  dbutils.fs.mount(
    source=f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net",
    mount_point=mount_point,
    extra_configs={
      f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": connect_str
    }
  )
  print(f"Mounted {mount_point}")
else:
  print(f"{mount_point} is already mounted")

/mnt/bees is already mounted


In [0]:
gold_df = spark.read.parquet("/mnt/bees/silver/breweries_silver")

In [0]:
gold_df = gold_df.groupBy("state", "brewery_type").agg(count("*").alias("count")).orderBy("count", ascending=False)

In [0]:
gold_df.display()

state,brewery_type,count
California,micro,461
Washington,micro,278
California,brewpub,249
New York,micro,224
Colorado,micro,221
Michigan,brewpub,199
Texas,micro,196
Florida,micro,190
North Carolina,micro,180
Ohio,micro,163


In [0]:
gold_df.write.partitionBy("state").mode("overwrite").parquet("/mnt/bees/gold/breweries_gold")